#Trips and Users

Write a SQL query to find the cancellation rate of requests with unbanned users
(both client and driver must not be banned) each day between "2013-10-01" and "2013-10-03".
Round Cancellation Rate to two decimal points.

The cancellation rate is computed by dividing the number of canceled (by client or driver)
requests with unbanned users by the total number of requests with unbanned users on that day.

In [0]:
%sql
-- Switch to my Catalog
USE CATALOG workspace;

-- Create schema if not exists
CREATE SCHEMA IF NOT EXISTS sql_pyspark_practice;

-- Use this schema
USE sql_pyspark_practice;

In [0]:
%sql
Create table  Trips (id int, client_id int, driver_id int, city_id int, status varchar(50), request_at varchar(50));

Create table Users (users_id int, banned varchar(50), role varchar(50));

insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('1', '1', '10', '1', 'completed', '2013-10-01');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('2', '2', '11', '1', 'cancelled_by_driver', '2013-10-01');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('3', '3', '12', '6', 'completed', '2013-10-01');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('4', '4', '13', '6', 'cancelled_by_client', '2013-10-01');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('5', '1', '10', '1', 'completed', '2013-10-02');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('6', '2', '11', '6', 'completed', '2013-10-02');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('7', '3', '12', '6', 'completed', '2013-10-02');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('8', '2', '12', '12', 'completed', '2013-10-03');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('9', '3', '10', '12', 'completed', '2013-10-03');
insert into Trips (id, client_id, driver_id, city_id, status, request_at) values ('10', '4', '13', '12', 'cancelled_by_driver', '2013-10-03');


insert into Users (users_id, banned, role) values ('1', 'No', 'client');
insert into Users (users_id, banned, role) values ('2', 'Yes', 'client');
insert into Users (users_id, banned, role) values ('3', 'No', 'client');
insert into Users (users_id, banned, role) values ('4', 'No', 'client');
insert into Users (users_id, banned, role) values ('10', 'No', 'driver');
insert into Users (users_id, banned, role) values ('11', 'No', 'driver');
insert into Users (users_id, banned, role) values ('12', 'No', 'driver');
insert into Users (users_id, banned, role) values ('13', 'No', 'driver');

In [0]:
%sql
select * from trips;

In [0]:
%sql
select * from users;

In [0]:
%sql

SELECT 
    request_at,
    ROUND(
        COUNT(CASE WHEN status IN ('cancelled_by_client','cancelled_by_driver') THEN 1 END)
        / COUNT(*) * 100,
    2) AS cancellation_rate
FROM Trips t
JOIN Users c ON t.client_id = c.users_id
JOIN Users d ON t.driver_id = d.users_id
WHERE c.banned = 'No' 
  AND d.banned = 'No'
  AND request_at BETWEEN '2013-10-01' AND '2013-10-03'
GROUP BY request_at;


In [0]:
from pyspark.sql import functions as F
trips_df = spark.table("trips")
users_df = spark.table("users")
# Alias Users dataframe for client and driver
clients_df = users_df.alias("c")
drivers_df = users_df.alias("d")
trips = trips_df.alias("t")

# Join trips with client and driver users
joined = (
    trips
    .join(clients_df, F.col("t.client_id") == F.col("c.users_id"))
    .join(drivers_df, F.col("t.driver_id") == F.col("d.users_id"))
)

# Filter: non-banned client & driver, date range filter
filtered = (
    joined
    .where(F.col("c.banned") == "No")
    .where(F.col("d.banned") == "No")
    .where((F.col("t.request_at") >= "2013-10-01") & (F.col("t.request_at") <= "2013-10-03"))
)

# Aggregate by date
result = (
    filtered
    .groupBy("t.request_at")
    .agg(
        # Count of cancelled trips
        F.sum(
            F.when(
                F.col("t.status").isin("cancelled_by_client", "cancelled_by_driver"), 
                1
            ).otherwise(0)
        ).alias("cancelled_trips"),
        # Total trips
        F.count(F.lit(1)).alias("total_trips")
    )
    # Compute cancellation rate and round
    .withColumn(
        "cancellation_rate",
        F.round(F.col("cancelled_trips") / F.col("total_trips") * 100, 2)
    )
    .select(
        F.col("t.request_at").alias("request_at"),
        "cancellation_rate"
    )
    .orderBy("request_at")
)

display(result)

## 1–3️⃣ What was the original question?

### Given SQL:

```sql
SELECT 
    request_at,
    ROUND(
        COUNT(CASE WHEN status IN ('cancelled_by_client','cancelled_by_driver') THEN 1 END)
        / COUNT(*) * 100,
    2) AS cancellation_rate
FROM Trips t
JOIN Users c ON t.client_id = c.users_id
JOIN Users d ON t.driver_id = d.users_id
WHERE c.banned = 'No' 
  AND d.banned = 'No'
  AND request_at BETWEEN '2013-10-01' AND '2013-10-03'
GROUP BY request_at;
```

From this, we can reconstruct something like this (this is essentially the original LeetCode “Trips and Users” question):

### Reconstructed Problem (in plain English)

We have two tables:

1. **Trips**

   * Contains ride requests between clients and drivers.
   * Has columns like:

     * `client_id` – which user requested the ride
     * `driver_id` – which user drove the ride
     * `status` – status of the trip (e.g., `'completed'`, `'cancelled_by_driver'`, `'cancelled_by_client'`)
     * `request_at` – the date the trip was requested

2. **Users**

   * Contains information about each user (both drivers and clients).
   * Has columns like:

     * `users_id` – unique user ID
     * `banned` – whether the user is banned (`'Yes'` / `'No'`)
     * `role` – `'client'` or `'driver'`

**Question:**

> For each day between **2013-10-01** and **2013-10-03** (inclusive),
> calculate the **cancellation rate of trips** made by **non-banned clients and non-banned drivers**.
>
> The **cancellation rate** for a date is:
>
> [
> \text{cancellation rate} = \frac{\text{number of cancelled trips on that date}}{\text{total trips on that date}} \times 100
> ]
>
> A trip is considered **cancelled** if `status` is `'cancelled_by_client'` or `'cancelled_by_driver'`.
> Return the result as:
>
> * `request_at` (the date)
> * `cancellation_rate` (rounded to 2 decimal places)

---

## 4️⃣ Why does this problem matter? What concepts is it testing?

This question is testing core **data analysis + SQL aggregation** skills:

1. **JOINs and table relationships**

   * You must realize that `Trips` references `Users` twice: once as client, once as driver.
   * You need to join the `Users` table **twice** with different aliases.

2. **Filtering rows by conditions**

   * Exclude banned users (`banned = 'No'` for both client and driver).
   * Restrict to a **date range** (`request_at BETWEEN '2013-10-01' AND '2013-10-03'`).

3. **Conditional aggregation**

   * Using `COUNT(CASE WHEN ... THEN 1 END)` to count only **cancelled** trips.
   * Using `COUNT(*)` for total trips.

4. **Grouping**

   * Grouping by date (`GROUP BY request_at`) to calculate metrics **per day**.

5. **Numeric computation**

   * Dividing one aggregate by another and multiplying by 100.
   * Using `ROUND(..., 2)` to round to 2 decimal places.
   * Awareness of integer vs decimal division.

These are exactly the kind of skills you need for **analytics interviews**, **LeetCode SQL**, and **real-world reporting queries**.

---

## 5️⃣ How to think about solving this type of problem (general methodology)

When you get a problem like:

> “Compute some rate/percentage per day for some subset of users/rows…”

You can think like this:

1. **Understand the tables and relationships**

   * Which table contains the events (trips)? → `Trips`
   * Which table contains attributes about users? → `Users`
   * How are they linked? → `Trips.client_id = Users.users_id`, `Trips.driver_id = Users.users_id`

2. **Clarify who/what to include**

   * Only trips with non-banned clients and drivers.
   * Only trips within a date range.
   * Only certain statuses count as "cancelled".

3. **Decide grouping keys**

   * What is the “per X” in the question?
     → Here it’s **per day** = group by `request_at`.

4. **Decide metrics to compute**

   * **Total trips per day**: `COUNT(*)`
   * **Cancelled trips per day**: `COUNT(CASE WHEN status IN (...) THEN 1 END)`
   * **Rate**: `cancelled / total * 100`, rounded.

5. **Structure the query**

   * FROM base table (Trips)
   * JOIN the tables needed (Users twice)
   * WHERE filters
   * GROUP BY date
   * SELECT date + computed metric

Once you internalize this pattern, you can reuse it for:

* completion rate
* click-through rate
* churn rate
* conversion rate
  …anything that is “number of X / total Y per group”.

---

## 6️⃣ Break the SQL solution into steps

Let’s dissect the SQL:

```sql
SELECT 
    request_at,
    ROUND(
        COUNT(CASE WHEN status IN ('cancelled_by_client','cancelled_by_driver') THEN 1 END)
        / COUNT(*) * 100,
    2) AS cancellation_rate
FROM Trips t
JOIN Users c ON t.client_id = c.users_id
JOIN Users d ON t.driver_id = d.users_id
WHERE c.banned = 'No' 
  AND d.banned = 'No'
  AND request_at BETWEEN '2013-10-01' AND '2013-10-03'
GROUP BY request_at;
```

### Step-by-step:

#### Step 1: FROM and JOINs

```sql
FROM Trips t
JOIN Users c ON t.client_id = c.users_id
JOIN Users d ON t.driver_id = d.users_id
```

* `Trips t`: base table, one row per trip.
* `Users c`: join to get info about the **client** (`c` = client).
* `Users d`: join to get info about the **driver** (`d` = driver).
* We now have each trip with:

  * trip data (`t.*`)
  * client attributes (`c.*`)
  * driver attributes (`d.*`)

#### Step 2: Filter banned users and date range

```sql
WHERE c.banned = 'No' 
  AND d.banned = 'No'
  AND request_at BETWEEN '2013-10-01' AND '2013-10-03'
```

* `c.banned = 'No'` → only trips where the **client** is not banned.
* `d.banned = 'No'` → only trips where the **driver** is not banned.
* `request_at BETWEEN '2013-10-01' AND '2013-10-03'`
  → only trips requested within those 3 days.

After this WHERE, the rows we’re grouping are exactly the trips we care about.

#### Step 3: Grouping by date

```sql
GROUP BY request_at;
```

* We want one row **per day**.
* So for each unique `request_at` (date), we compute aggregates.

#### Step 4: Counting cancelled vs total trips

Inside `SELECT`:

```sql
COUNT(CASE WHEN status IN ('cancelled_by_client','cancelled_by_driver') THEN 1 END)
```

* `CASE` is used only to count certain rows.
* If `status` is one of the cancelled statuses → returns `1`, so `COUNT(1)` counts it.
* If status is something else (e.g., `'completed'`), `CASE` returns `NULL`; `COUNT(NULL)` ignores it.
* So this gives: **cancelled trip count per day**.

```sql
COUNT(*) 
```

* Counts **all trips** (non-filtered, within this group).
* This is the **total trip count per day**.

#### Step 5: Computing the percentage and rounding

```sql
ROUND(
    COUNT(CASE WHEN status IN ('cancelled_by_client','cancelled_by_driver') THEN 1 END)
    / COUNT(*) * 100,
2) AS cancellation_rate
```

* Numerator: number of cancelled trips per day.
* Denominator: total trips per day.
* Multiply by 100 to make it a percentage.
* `ROUND(..., 2)` rounds to 2 decimal places.
* aliased as `cancellation_rate`.

> ⚠️ Practical detail: in some SQL engines, `COUNT(...) / COUNT(*)` will do **integer division**.
> To be safe you might write:
>
> ```sql
> 1.0 * COUNT(...) / COUNT(*) * 100
> ```
>
> or cast one side to `DECIMAL`. But conceptually, your logic is correct.

---

## 7️⃣ Rewrite the SQL in a cleaner, commented way

```sql
-- Calculate daily cancellation rate for trips between 2013-10-01 and 2013-10-03
-- considering only trips where BOTH client and driver are not banned.
SELECT
    t.request_at,  -- date on which the trip was requested

    -- Cancellation rate = (cancelled trips / total trips) * 100
    ROUND(
        100.0 * 
        COUNT(
            CASE 
                WHEN t.status IN ('cancelled_by_client', 'cancelled_by_driver') 
                THEN 1 
            END
        ) / COUNT(*),
        2
    ) AS cancellation_rate
FROM Trips t
    -- Join users table as client
    JOIN Users c 
        ON t.client_id = c.users_id
    -- Join users table as driver
    JOIN Users d 
        ON t.driver_id = d.users_id
WHERE 
    c.banned = 'No'                -- client not banned
    AND d.banned = 'No'            -- driver not banned
    AND t.request_at BETWEEN '2013-10-01' AND '2013-10-03'
GROUP BY 
    t.request_at
ORDER BY 
    t.request_at;
```

---

## 8️⃣ Equivalent PySpark DataFrame code

Assume you already have two DataFrames:

* `trips_df` for `Trips`
* `users_df` for `Users`

We’ll do the same logic in PySpark:

```python
from pyspark.sql import functions as F

# Alias Users dataframe for client and driver
clients_df = users_df.alias("c")
drivers_df = users_df.alias("d")
trips = trips_df.alias("t")

# Join trips with client and driver users
joined = (
    trips
    .join(clients_df, F.col("t.client_id") == F.col("c.users_id"))
    .join(drivers_df, F.col("t.driver_id") == F.col("d.users_id"))
)

# Filter: non-banned client & driver, date range filter
filtered = (
    joined
    .where(F.col("c.banned") == "No")
    .where(F.col("d.banned") == "No")
    .where((F.col("t.request_at") >= "2013-10-01") & (F.col("t.request_at") <= "2013-10-03"))
)

# Aggregate by date
result = (
    filtered
    .groupBy("t.request_at")
    .agg(
        # Count of cancelled trips
        F.sum(
            F.when(
                F.col("t.status").isin("cancelled_by_client", "cancelled_by_driver"), 
                1
            ).otherwise(0)
        ).alias("cancelled_trips"),
        # Total trips
        F.count(F.lit(1)).alias("total_trips")
    )
    # Compute cancellation rate and round
    .withColumn(
        "cancellation_rate",
        F.round(F.col("cancelled_trips") / F.col("total_trips") * 100, 2)
    )
    .select(
        F.col("t.request_at").alias("request_at"),
        "cancellation_rate"
    )
    .orderBy("request_at")
)
```

---

## 9️⃣ Explain the PySpark code line by line

```python
from pyspark.sql import functions as F
```

* Import PySpark SQL functions with alias `F` (common convention).

```python
clients_df = users_df.alias("c")
drivers_df = users_df.alias("d")
trips = trips_df.alias("t")
```

* We create **aliases** for the same `users_df` DataFrame:

  * `"c"` for clients
  * `"d"` for drivers
* Alias `trips_df` as `"t"` to mimic SQL table aliases.

```python
joined = (
    trips
    .join(clients_df, F.col("t.client_id") == F.col("c.users_id"))
    .join(drivers_df, F.col("t.driver_id") == F.col("d.users_id"))
)
```

* Join `Trips` with `Users` for client, then join again for driver.
* Same logic as SQL `JOIN Users c ON ...` and `JOIN Users d ON ...`.

```python
filtered = (
    joined
    .where(F.col("c.banned") == "No")
    .where(F.col("d.banned") == "No")
    .where((F.col("t.request_at") >= "2013-10-01") & (F.col("t.request_at") <= "2013-10-03"))
)
```

* Add filters:

  * client not banned
  * driver not banned
  * request_at between the two dates
* Equivalent to SQL `WHERE` clause.

```python
result = (
    filtered
    .groupBy("t.request_at")
    .agg(
        F.sum(
            F.when(
                F.col("t.status").isin("cancelled_by_client", "cancelled_by_driver"), 
                1
            ).otherwise(0)
        ).alias("cancelled_trips"),
        F.count(F.lit(1)).alias("total_trips")
    )
```

* `groupBy("t.request_at")` → group rows by `request_at` date.
* Inside `.agg(...)` we calculate:

  * `cancelled_trips`:

    * `F.when(condition, 1).otherwise(0)` gives **1 or 0 per row**.
    * `F.sum(...)` totals them per group.
  * `total_trips`:

    * `F.count(F.lit(1))` counts all rows in the group (like `COUNT(*)`).

```python
    .withColumn(
        "cancellation_rate",
        F.round(F.col("cancelled_trips") / F.col("total_trips") * 100, 2)
    )
```

* Create a new column `cancellation_rate` = (cancelled / total * 100), rounded to 2 decimals.

```python
    .select(
        F.col("t.request_at").alias("request_at"),
        "cancellation_rate"
    )
    .orderBy("request_at")
)
```

* Select only the relevant columns, rename the date column nicely, and order by date.

At the end, `result` is a DataFrame equivalent to the SQL result.

---

## 🔟 How the thinking differs between SQL and PySpark

**SQL mindset:**

* You describe *what* you want, not *how* to compute it.
* One big declarative statement:

  * FROM → JOIN → WHERE → GROUP BY → SELECT.
* The DB engine figures out the optimal execution plan.

**PySpark mindset:**

* You chain **transformations** on DataFrames.
* Each step is a transformation: `join`, `where`, `groupBy`, `agg`, `withColumn`, `select`.
* You explicitly build a **pipeline** of operations.
* Execution is lazy: nothing runs until you perform an action (`show`, `collect`, `write` etc.).

Conceptually they are the same operations, but:

* SQL = one declarative query.
* PySpark = sequence of transformations that logically mirror the SQL but in an API style.

Once you can read/write the SQL clearly, translating to PySpark is mostly mechanic.

---

## 1️⃣1️⃣ Hints to solve this (without looking at the solution)

### 🟢 Basic hints

1. **Hint 1 (tables):**
   Identify the table with trips and the table with user info (banned or not). Ask: “How do I know if a trip’s client or driver is banned?”

2. **Hint 2 (double join):**
   You need to use the **Users** table twice: once as client, once as driver. This will require **two joins** with different aliases.

3. **Hint 3 (grouping):**
   The answer is “per day”, so think about grouping by the **date column** and using aggregate functions.

### 🔵 Advanced hints

4. **Hint 4 (conditional count):**
   To count only cancelled trips, you don’t filter them out in the WHERE (or you’d lose other statuses). Instead, use something like `COUNT(CASE WHEN ... THEN 1 END)`.

5. **Hint 5 (rate formula):**
   The cancellation rate is `cancelled_trips / total_trips * 100`. Both come from aggregates in the same group.

6. **Hint 6 (filters placement):**
   Be careful not to put the “cancelled” condition in `WHERE`, or you’ll only keep cancelled trips and lose the denominator. The cancelled condition belongs inside `CASE` in the aggregate.

---

## 1️⃣2️⃣ Final SQL and PySpark solutions (clean)

### ✅ Final SQL

```sql
SELECT
    t.request_at,
    ROUND(
        100.0 * 
        COUNT(
            CASE 
                WHEN t.status IN ('cancelled_by_client', 'cancelled_by_driver') 
                THEN 1 
            END
        ) / COUNT(*),
        2
    ) AS cancellation_rate
FROM Trips t
JOIN Users c 
    ON t.client_id = c.users_id
JOIN Users d 
    ON t.driver_id = d.users_id
WHERE 
    c.banned = 'No'
    AND d.banned = 'No'
    AND t.request_at BETWEEN '2013-10-01' AND '2013-10-03'
GROUP BY 
    t.request_at
ORDER BY 
    t.request_at;
```

---

### ✅ Final PySpark

```python
from pyspark.sql import functions as F

# Aliases
trips = trips_df.alias("t")
clients_df = users_df.alias("c")
drivers_df = users_df.alias("d")

# Join trips with client and driver users
joined = (
    trips
    .join(clients_df, F.col("t.client_id") == F.col("c.users_id"))
    .join(drivers_df, F.col("t.driver_id") == F.col("d.users_id"))
)

# Filter by non-banned users and date range
filtered = (
    joined
    .where(F.col("c.banned") == "No")
    .where(F.col("d.banned") == "No")
    .where(
        (F.col("t.request_at") >= "2013-10-01") &
        (F.col("t.request_at") <= "2013-10-03")
    )
)

# Group and compute cancellation rate
result = (
    filtered
    .groupBy("t.request_at")
    .agg(
        F.sum(
            F.when(
                F.col("t.status").isin("cancelled_by_client", "cancelled_by_driver"),
                1
            ).otherwise(0)
        ).alias("cancelled_trips"),
        F.count(F.lit(1)).alias("total_trips")
    )
    .withColumn(
        "cancellation_rate",
        F.round(F.col("cancelled_trips") / F.col("total_trips") * 100, 2)
    )
    .select(
        F.col("t.request_at").alias("request_at"),
        "cancellation_rate"
    )
    .orderBy("request_at")
)
```

---